In [ ]:
!pip install tesseract pytesseract opencv-python tesserocr  python-Levenshtein gspread oauth2client

In [1]:
from PIL import Image
import cv2
import pytesseract
from pathlib import Path
import glob
import numpy as np
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials

pytesseract.pytesseract.tesseract_cmd = ".\\tesseract\\tesseract.exe"

In [2]:
IMG_FOLDER_PATH = r'C:\Users\Azazel\Desktop\projekte\fallout\screenshots' + "\\"
effects_path = ".\\effects.json"
weapon_types_path = ".\\weapon types.json"
google_api_file_path = ".\\FalloutReader-979156523830.json"
sheet_name = "testsheet"

In [18]:
images = [x for x in glob.glob(IMG_FOLDER_PATH + "*.png")]
descriptions = []
weapon_types = []
with open (effects_path,encoding="utf8") as f:
    j = json.load(f)
    descriptions = list(j.items())
with open (weapon_types_path,encoding="utf8") as f:
    j = json.load(f)
    weapon_types = list(j.items())

In [42]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(google_api_file_path, scope)
client = gspread.authorize(creds)
sheet = client.open(sheet_name).sheet1
ids = sheet.col_values(1)[1:]

In [19]:
def expand_image(img):
    old_size = img.size
    new_size = (100, 100)
    new_im = Image.new("RGB", new_size, (255, 255, 255))
    new_im.paste(img, (int((new_size[0]-old_size[0])/2),
                          int((new_size[1]-old_size[1])/2)))
    return new_im

In [20]:
def read_int(img, fl=False):
    img = expand_image(img)
    return pytesseract.image_to_string(img, lang = 'deu', config='--psm 6 -c tessedit_char_whitelist=0123456789' + (".," if fl  else "")).strip()

In [21]:
from matplotlib.pyplot import imshow
import numpy as np
def read_statistics(cropped_img):
    level = cropped_img.crop((70,40,110,85))
    dmg = cropped_img.crop((236,110,290,160))
    firerate = cropped_img.crop((210,180,290,230))
    _range = cropped_img.crop((210,220,290,260))
    precision = cropped_img.crop((236,260,285,295))
    weight = cropped_img.crop((215,290,290,340))
    level = read_int(level)
    dmg = read_int(dmg)
    firerate = read_int(firerate)
    _range =read_int(_range)
    precision = read_int(precision)
    #weight =  read_int(weight)
    #weight = float(int(weight.replace(" ", ""))/10)
    return level,dmg,firerate,_range,precision,weight


In [22]:
def read_sentence(crop):
    return pytesseract.image_to_string(crop, lang = 'deu', config=r'-c load_system_dawg=false -c load_freq_dawg=false --user-words C:\Users\Azazel\Desktop\projekte\fallout\tesseract\tessdata\deu.user-words')

In [23]:
def read_name(cropped_image):
    name_as_pixels = np.array(cropped_image)
    x,y, c = name_as_pixels.shape
    for i in range(0,x):
        for j in range(0,y):
            pixel = name_as_pixels[i][j]
            diff = np.absolute(pixel - [244, 202,  91])
            if np.all(diff < 5):
                name_as_pixels[i][j] = [0,0,0]
            else:
                name_as_pixels[i][j] = [255,255,255]
    clean_name = Image.fromarray(name_as_pixels)
    line = read_sentence(clean_name)
    for n,v in weapon_types:
        if n in line:
            return v
    return "not recognized"

In [24]:
from Levenshtein import distance
from tqdm import tqdm_notebook as tqdm
import PIL.ImageOps 
results = []
for image in tqdm(images):
    _id = Path(image).stem.replace("ScreenShot", "")
    img = Image.open(image)
    name = img.crop((500,40,1330,85))
    thresh = 225
    fn = lambda x : 255 if x > thresh else 0
    img = img.convert('L').point(fn, mode='1')
    img = PIL.ImageOps.invert(img.convert('L'))
    statistics = img.crop((310,520,600,930))
    level = read_statistics(statistics)[0]
    weapon_type = read_name(name)
    effects1 = img.crop((520,75,1360,115))
    effects2 = img.crop((520,110,1360,145))
    effects3 = img.crop((520,145,1360,180))
    sentences = [read_sentence(effects1),read_sentence(effects2),read_sentence(effects3)]
    sentences = [s for s in sentences if len(s.strip()) > 0]
    effects = []
    for s in sentences:
        for d,v in descriptions:
            edit_dist = distance(d, s)
            if edit_dist < 5:
                effects.append(v)
    results.append((_id,level,len(effects),weapon_type, effects))

In [29]:
#print(sheet.get_all_values())

[['test', 'test2']]


In [31]:
for i,(_id,level,stars,weapon_type, effects) in enumerate(results,2):
    ef = ", ".join(effects)
    row = [_id, level, len(effects), weapon_type, ef]
    row = [str(c) for c in row]
    sheet.insert_row(row, i)

AttributeError: 'list' object has no attribute 'setBackgrounds'

In [41]:
def delete_rows():
    row_count = len(sheet.col_values(1))
    for i in range(2,row_count+3):
        sheet.delete_row(i)

# TODO
* Ids aus sheet auslesen und bei exists nicht erneut lesen
* Berechnung parallelisieren
